# Classical Shadows: A Deep Dive into the Implementation

This notebook walks through **every step** of QuartumSE's classical shadows implementation, explaining both the theory and the code.

## What You'll Learn

1. **Theory**: The mathematical foundation of classical shadows
2. **Step 1**: Random basis selection (local Clifford sampling)
3. **Step 2**: Circuit generation with measurement rotations
4. **Step 3**: Measurement execution and data collection
5. **Step 4**: Shadow reconstruction via inverse channel
6. **Step 5**: Observable estimation with the 3^k scaling factor
7. **Step 6**: Confidence intervals and variance bounds
8. **Bonus**: Noise-aware shadows (v1) with MEM

## Reference

Huang, Kueng, Preskill (2020) - "Predicting Many Properties of a Quantum System" ([arXiv:2002.08953](https://arxiv.org/abs/2002.08953))

---

## Part 0: The Theory Behind Classical Shadows

### The Core Idea

Classical shadows create a **classical description** of a quantum state that can be used to estimate **any observable** later.

### The Protocol

1. **Prepare** the quantum state $\rho$
2. **Apply** a random unitary $U$ from an ensemble (e.g., local Cliffords)
3. **Measure** in computational basis, get outcome $|b\rangle$
4. **Invert** to get a "snapshot": $\hat{\rho} = \mathcal{M}^{-1}(U^\dagger |b\rangle\langle b| U)$

### The Key Insight

The **inverse channel** $\mathcal{M}^{-1}$ ensures that the average snapshot equals the true state:

$$\mathbb{E}[\hat{\rho}] = \rho$$

### For Random Local Cliffords

When using random single-qubit Clifford gates (which rotate to X, Y, or Z basis):

- The inverse channel for a single qubit is: $\hat{\rho}_q = 3|b_q\rangle\langle b_q| - I$
- For n qubits: $\hat{\rho} = \bigotimes_{q=1}^{n} \hat{\rho}_q$

### Estimating Observables

To estimate $\langle O \rangle = \text{Tr}(O\rho)$:

$$\hat{o} = \frac{1}{M} \sum_{i=1}^{M} \text{Tr}(O \hat{\rho}_i)$$

For **Pauli observables**, this simplifies dramatically:
- If the measurement basis matches the Pauli, we get a signal
- If it doesn't match, that shadow contributes 0

---

## Part 1: Setup and Imports

In [ ]:
# Standard imports
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

# Add QuartumSE to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

# QuartumSE imports - we'll use these AND manually implement steps
from quartumse.shadows.config import ShadowConfig, ShadowVersion
from quartumse.shadows.core import Observable, ShadowEstimate
from quartumse.shadows.v0_baseline import RandomLocalCliffordShadows

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Setup complete!")

## Part 2: Create Our Test State

We'll use a simple Bell state for clarity: $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$

### Known Expectation Values:
- $\langle ZI \rangle = 0$ (first qubit in superposition)
- $\langle IZ \rangle = 0$ (second qubit in superposition)
- $\langle ZZ \rangle = +1$ (perfect correlation)
- $\langle XX \rangle = +1$ (perfect correlation in X basis)
- $\langle YY \rangle = -1$ (anti-correlation in Y basis)

In [ ]:
# Create Bell state circuit
def create_bell_state():
    qc = QuantumCircuit(2)
    qc.h(0)       # Hadamard on qubit 0
    qc.cx(0, 1)   # CNOT: control=0, target=1
    return qc

bell_circuit = create_bell_state()

print("Bell State Preparation Circuit:")
print(bell_circuit)

# Ground truth expectation values
GROUND_TRUTH = {
    "ZI": 0.0,
    "IZ": 0.0,
    "ZZ": 1.0,
    "XX": 1.0,
    "YY": -1.0,
    "XY": 0.0,
    "YX": 0.0,
}

print("\nGround Truth Expectation Values:")
for obs, val in GROUND_TRUTH.items():
    print(f"  <{obs}> = {val:+.1f}")

---

## Step 1: Random Basis Selection

The first step is to choose **random measurement bases** for each qubit.

### The Three Bases

| Index | Basis | Gate to Apply | What We Measure |
|-------|-------|---------------|------------------|
| 0 | Z | Identity (I) | Computational basis |
| 1 | X | Hadamard (H) | Superposition basis |
| 2 | Y | S†H | Y eigenbasis |

### Why These Gates?

- **Z basis (index 0)**: No rotation needed - just measure directly
- **X basis (index 1)**: H rotates X eigenstates to Z eigenstates
- **Y basis (index 2)**: S†H rotates Y eigenstates to Z eigenstates

### Implementation in QuartumSE

From `v0_baseline.py`:
```python
self.basis_gates = {
    0: lambda qc, q: None,           # Z basis (Identity)
    1: lambda qc, q: qc.h(q),         # X basis (Hadamard)
    2: lambda qc, q: [qc.sdg(q), qc.h(q)],  # Y basis (S†H)
}
```

In [ ]:
# Manual implementation of random basis selection
NUM_SHADOWS = 10  # Small number for demonstration
NUM_QUBITS = 2

rng = np.random.default_rng(RANDOM_SEED)

# Sample random bases: 0=Z, 1=X, 2=Y
measurement_bases = rng.integers(0, 3, size=(NUM_SHADOWS, NUM_QUBITS))

print("Random Measurement Bases (0=Z, 1=X, 2=Y):")
print("=" * 50)
basis_names = {0: 'Z', 1: 'X', 2: 'Y'}

for shadow_idx in range(NUM_SHADOWS):
    bases = measurement_bases[shadow_idx]
    basis_str = "".join(basis_names[b] for b in bases)
    print(f"Shadow {shadow_idx}: qubit bases = {bases} -> measuring in {basis_str} basis")

# Count basis distribution
print("\nBasis Distribution:")
for basis_idx, basis_name in basis_names.items():
    count = (measurement_bases == basis_idx).sum()
    pct = 100 * count / measurement_bases.size
    print(f"  {basis_name}: {count} times ({pct:.1f}%)")

print("\nNote: With uniform random sampling, each basis appears ~33% of the time.")

---

## Step 2: Generate Measurement Circuits

For each shadow, we:
1. Copy the state preparation circuit
2. Add rotation gates based on the selected bases
3. Add measurements

### The Gate Mapping

```
If basis == 0 (Z): Do nothing
If basis == 1 (X): Apply H
If basis == 2 (Y): Apply S†, then H
```

### Why This Works

- To measure in X basis: $H|+\rangle = |0\rangle$, $H|-\rangle = |1\rangle$
- To measure in Y basis: $HS^\dagger|+i\rangle = |0\rangle$, $HS^\dagger|-i\rangle = |1\rangle$

In [ ]:
def apply_basis_rotation(qc: QuantumCircuit, qubit: int, basis: int) -> None:
    """
    Apply rotation to measure in the specified basis.
    
    basis=0 (Z): No rotation
    basis=1 (X): Apply H
    basis=2 (Y): Apply S†, then H
    """
    if basis == 0:  # Z basis
        pass  # No rotation needed
    elif basis == 1:  # X basis
        qc.h(qubit)
    elif basis == 2:  # Y basis
        qc.sdg(qubit)  # S-dagger
        qc.h(qubit)


def generate_shadow_circuit(base_circuit: QuantumCircuit, bases: np.ndarray) -> QuantumCircuit:
    """
    Generate a single shadow measurement circuit.
    
    Args:
        base_circuit: State preparation circuit
        bases: Array of basis indices for each qubit
    
    Returns:
        Circuit with rotations and measurements
    """
    qc = base_circuit.copy()
    
    # Apply basis rotations
    for qubit_idx, basis in enumerate(bases):
        apply_basis_rotation(qc, qubit_idx, basis)
    
    # Add measurements
    qc.measure_all()
    
    return qc


# Generate circuits for our shadows
shadow_circuits = []
for shadow_idx in range(NUM_SHADOWS):
    bases = measurement_bases[shadow_idx]
    circuit = generate_shadow_circuit(bell_circuit, bases)
    shadow_circuits.append(circuit)

print(f"Generated {len(shadow_circuits)} shadow circuits")

# Show a few example circuits
print("\n" + "="*60)
print("Example Shadow Circuits:")
print("="*60)

for i in [0, 1, 2]:
    bases = measurement_bases[i]
    basis_str = "".join(basis_names[b] for b in bases)
    print(f"\nShadow {i} - Measuring in {basis_str} basis:")
    print(shadow_circuits[i])

---

## Step 3: Execute Circuits and Collect Measurements

Now we run each circuit **once** (single shot per circuit) and record the outcome.

### Why Single Shot Per Circuit?

Each shadow is an independent random measurement. Running with `shots=1` gives us:
- One bitstring outcome per shadow
- Maximum information diversity across different measurement bases

### The Data We Collect

For each shadow $i$:
- `measurement_bases[i]`: Which basis was measured for each qubit
- `measurement_outcomes[i]`: The binary outcome (0 or 1) for each qubit

In [ ]:
# Execute circuits
backend = AerSimulator(seed_simulator=RANDOM_SEED)

# Transpile circuits
transpiled_circuits = transpile(shadow_circuits, backend=backend)

# Run with shots=1 per circuit
job = backend.run(transpiled_circuits, shots=1)
result = job.result()

# Extract measurement outcomes
measurement_outcomes = []

for circuit_idx in range(NUM_SHADOWS):
    counts = result.get_counts(circuit_idx)
    # Get the single bitstring outcome
    bitstring = list(counts.keys())[0].replace(" ", "")
    # Convert to array (note: Qiskit bitstrings are reversed)
    outcomes = np.array([int(b) for b in bitstring[::-1]], dtype=int)
    measurement_outcomes.append(outcomes)

measurement_outcomes = np.array(measurement_outcomes)

print("Measurement Results:")
print("=" * 70)
print(f"{'Shadow':<8} {'Bases':<10} {'Basis Names':<12} {'Outcomes':<10} {'Bitstring'}")
print("-" * 70)

for i in range(NUM_SHADOWS):
    bases = measurement_bases[i]
    outcomes = measurement_outcomes[i]
    basis_str = "".join(basis_names[b] for b in bases)
    bitstring = "".join(str(b) for b in outcomes)
    print(f"{i:<8} {str(bases):<10} {basis_str:<12} {str(outcomes):<10} |{bitstring}>")

print("\nData shapes:")
print(f"  measurement_bases: {measurement_bases.shape}")
print(f"  measurement_outcomes: {measurement_outcomes.shape}")

---

## Step 4: The Inverse Channel (Shadow Reconstruction)

This is the **key mathematical step** that makes classical shadows work.

### The Problem

When we measure in a random basis and get outcome $|b\rangle$, we have:
$$\text{measured state} = U^\dagger |b\rangle\langle b| U$$

But this is **biased** - averaging these directly doesn't give us $\rho$.

### The Solution: Inverse Channel

Apply a correction factor to "undo" the bias:

For a **single qubit** with outcome $b \in \{0, 1\}$:
$$\hat{\rho}_q = 3|b\rangle\langle b| - I$$

In matrix form:
- If outcome = 0: $\hat{\rho} = 3\begin{pmatrix}1&0\\0&0\end{pmatrix} - I = \begin{pmatrix}2&0\\0&-1\end{pmatrix}$
- If outcome = 1: $\hat{\rho} = 3\begin{pmatrix}0&0\\0&1\end{pmatrix} - I = \begin{pmatrix}-1&0\\0&2\end{pmatrix}$

### Why Factor of 3?

The factor of 3 comes from inverting the "measurement channel":
- With 3 equally likely bases (X, Y, Z), the average measurement projects onto $\frac{1}{3}$ of the information
- Multiplying by 3 corrects for this

### For Multiple Qubits

The full snapshot is a tensor product:
$$\hat{\rho} = \bigotimes_{q=1}^{n} (3|b_q\rangle\langle b_q| - I)$$

In [ ]:
# Define the inverse channel matrices
def inverse_channel_single_qubit(outcome: int) -> np.ndarray:
    """
    Compute the inverse channel snapshot for a single qubit.
    
    Formula: rho_hat = 3|b><b| - I
    
    Args:
        outcome: 0 or 1
    
    Returns:
        2x2 matrix representing the snapshot
    """
    if outcome == 0:
        ket_b = np.array([[1], [0]])  # |0>
    else:
        ket_b = np.array([[0], [1]])  # |1>
    
    # |b><b|
    proj_b = ket_b @ ket_b.T
    
    # 3|b><b| - I
    snapshot = 3 * proj_b - np.eye(2)
    
    return snapshot


# Demonstrate the inverse channel
print("Inverse Channel Matrices:")
print("=" * 50)

print("\nOutcome = 0:")
print("  rho_hat = 3|0><0| - I =")
print(inverse_channel_single_qubit(0))

print("\nOutcome = 1:")
print("  rho_hat = 3|1><1| - I =")
print(inverse_channel_single_qubit(1))

print("\n" + "="*50)
print("Verification: Average of snapshots equals maximally mixed state")
print("="*50)
avg_snapshot = 0.5 * inverse_channel_single_qubit(0) + 0.5 * inverse_channel_single_qubit(1)
print("\nE[rho_hat] for uniform outcomes:")
print(avg_snapshot)
print("\nThis equals I/2 (maximally mixed), which is correct for a random qubit!")

---

## Step 5: Observable Estimation (The Core Algorithm)

Now comes the key part: estimating expectation values from our shadow data.

### The Estimator Formula

For a Pauli observable $P = P_{q_1} \otimes P_{q_2} \otimes \ldots$ (where each $P_q \in \{I, X, Y, Z\}$):

$$\langle P \rangle \approx \frac{1}{M} \sum_{i=1}^{M} \text{Tr}(P \cdot \hat{\rho}_i)$$

### The Clever Trick

For **random local Clifford** shadows, we don't need to compute full density matrices!

For each shadow $i$ and observable $P$:

1. **Check compatibility**: Did we measure in the right basis for each non-identity Pauli?
   - If $P_q = Z$, we need to have measured qubit $q$ in Z basis
   - If $P_q = X$, we need to have measured qubit $q$ in X basis
   - If $P_q = Y$, we need to have measured qubit $q$ in Y basis

2. **If compatible**: Compute $3^k \times \prod_{q \in \text{support}} (-1)^{b_q}$
   - $k$ = number of non-identity Paulis (the "support size")
   - $b_q$ = measurement outcome for qubit $q$
   - The $3^k$ factor comes from the inverse channel!

3. **If incompatible**: This shadow contributes 0 to the estimate

### Why This Works

- Pauli operators are eigenstates of the corresponding measurement basis
- $\text{Tr}(Z \cdot (3|b\rangle\langle b| - I)) = 3(1-2b) - 0 = 3 \times (\pm 1)$
- The trace over non-matching bases gives 0 (orthogonality)

In [ ]:
def estimate_pauli_single_shadow(
    shadow_idx: int,
    pauli_string: str,
    measurement_bases: np.ndarray,
    measurement_outcomes: np.ndarray,
    verbose: bool = False
) -> float:
    """
    Estimate Pauli expectation value from a single shadow.
    
    This implements the core classical shadows estimator.
    
    Args:
        shadow_idx: Index of the shadow
        pauli_string: e.g., "ZI", "XX", "YY"
        measurement_bases: Array of shape (num_shadows, num_qubits)
        measurement_outcomes: Array of shape (num_shadows, num_qubits)
        verbose: Print detailed computation steps
    
    Returns:
        Contribution from this shadow (may be 0 if incompatible)
    """
    # Map Pauli characters to required basis indices
    pauli_to_basis = {'X': 1, 'Y': 2, 'Z': 0, 'I': None}
    
    bases = measurement_bases[shadow_idx]
    outcomes = measurement_outcomes[shadow_idx]
    
    if verbose:
        print(f"  Shadow {shadow_idx}: bases={bases}, outcomes={outcomes}")
    
    expectation = 1.0
    support_size = 0  # Count non-identity Paulis
    
    for qubit_idx, pauli in enumerate(pauli_string):
        if pauli == 'I':
            continue  # Identity doesn't affect the result
        
        support_size += 1
        required_basis = pauli_to_basis[pauli]
        measured_basis = bases[qubit_idx]
        outcome = outcomes[qubit_idx]
        
        if verbose:
            print(f"    Qubit {qubit_idx}: Pauli={pauli}, need basis={required_basis}, got basis={measured_basis}")
        
        if measured_basis != required_basis:
            if verbose:
                print(f"    -> INCOMPATIBLE! Returning 0")
            return 0.0  # Incompatible measurement
        
        # Compatible: multiply by sign based on outcome
        # outcome=0 -> +1, outcome=1 -> -1
        sign = 1 - 2 * outcome
        expectation *= sign
        
        if verbose:
            print(f"    -> outcome={outcome}, sign={sign:+d}")
    
    # Apply the 3^k scaling factor from the inverse channel
    scaling_factor = 3 ** support_size
    result = scaling_factor * expectation
    
    if verbose:
        print(f"  -> support_size={support_size}, scaling=3^{support_size}={scaling_factor}, result={result:+.0f}")
    
    return result


# Demonstrate with a specific example
print("Detailed Estimation Example:")
print("=" * 60)
print("\nEstimating <ZZ> (expect +1 for Bell state):")
print()

for shadow_idx in range(min(5, NUM_SHADOWS)):
    result = estimate_pauli_single_shadow(
        shadow_idx, "ZZ", measurement_bases, measurement_outcomes, verbose=True
    )
    print()

In [ ]:
def estimate_observable_from_shadows(
    pauli_string: str,
    measurement_bases: np.ndarray,
    measurement_outcomes: np.ndarray,
    coefficient: float = 1.0
) -> dict:
    """
    Estimate observable using all shadow data.
    
    Returns:
        dict with expectation, variance, individual contributions
    """
    num_shadows = len(measurement_outcomes)
    
    # Compute contribution from each shadow
    contributions = []
    for i in range(num_shadows):
        contrib = estimate_pauli_single_shadow(
            i, pauli_string, measurement_bases, measurement_outcomes
        )
        contributions.append(contrib)
    
    contributions = np.array(contributions)
    
    # Mean estimator
    expectation = np.mean(contributions) * coefficient
    
    # Variance
    variance = np.var(contributions)
    
    # Count compatible shadows
    compatible_count = np.sum(contributions != 0)
    
    return {
        'expectation': expectation,
        'variance': variance,
        'std_error': np.sqrt(variance / num_shadows),
        'contributions': contributions,
        'compatible_shadows': compatible_count,
        'compatibility_rate': compatible_count / num_shadows,
    }


# Estimate all our target observables
print("Observable Estimation Results (Manual Implementation):")
print("=" * 80)
print(f"{'Observable':<12} {'Estimated':<12} {'Expected':<10} {'Error':<10} {'Compatible':<12} {'Contrib Dist'}")
print("-" * 80)

for obs_str, expected in GROUND_TRUTH.items():
    result = estimate_observable_from_shadows(
        obs_str, measurement_bases, measurement_outcomes
    )
    est = result['expectation']
    err = abs(est - expected)
    compat = result['compatible_shadows']
    compat_pct = 100 * result['compatibility_rate']
    
    # Show distribution of contributions
    contribs = result['contributions']
    nonzero = contribs[contribs != 0]
    if len(nonzero) > 0:
        contrib_str = f"mean={np.mean(nonzero):+.1f}"
    else:
        contrib_str = "all zero"
    
    print(f"{obs_str:<12} {est:>+10.4f}   {expected:>+8.1f}   {err:>8.4f}   {compat}/{NUM_SHADOWS} ({compat_pct:4.1f}%)  {contrib_str}")

print("\nNote: With only 10 shadows, estimates may be noisy. More shadows = better precision.")

---

## Step 6: Confidence Intervals and Variance Bounds

### Theoretical Variance Bound

For random local Clifford shadows, the variance of the estimator is bounded by:

$$\text{Var}[\hat{o}] \leq \frac{4^k}{M}$$

where:
- $k$ = support size (number of non-identity Paulis)
- $M$ = number of shadows

### Why $4^k$?

- Each non-identity Pauli can contribute $\pm 3$ when compatible
- The probability of being compatible is $1/3$ per qubit
- The variance of a single shadow's contribution is $O(9^k \cdot (1/3)^k) = O(3^k)$
- The theoretical bound is $4^k$ (slightly looser for simplicity)

### Confidence Intervals

Using the normal approximation (valid for large M):

$$\text{CI}_{95\%} = \bar{o} \pm 1.96 \cdot \frac{\sigma}{\sqrt{M}}$$

In [ ]:
from scipy import stats

def compute_confidence_interval(
    mean: float,
    variance: float,
    n_samples: int,
    confidence: float = 0.95
) -> tuple:
    """
    Compute confidence interval using normal approximation.
    """
    std_error = np.sqrt(variance / n_samples)
    z_score = stats.norm.ppf((1 + confidence) / 2)
    
    ci_lower = mean - z_score * std_error
    ci_upper = mean + z_score * std_error
    
    return (ci_lower, ci_upper)


def variance_bound(pauli_string: str, shadow_size: int) -> float:
    """
    Compute theoretical variance bound for a Pauli observable.
    
    Var <= 4^k / M, where k is the support size.
    """
    support_size = sum(1 for p in pauli_string if p != 'I')
    return (4 ** support_size) / shadow_size


print("Confidence Intervals and Variance Analysis:")
print("=" * 90)
print(f"{'Observable':<12} {'Support':<8} {'Var Bound':<12} {'Actual Var':<12} {'95% CI':<25} {'Contains Truth?'}")
print("-" * 90)

for obs_str, expected in GROUND_TRUTH.items():
    result = estimate_observable_from_shadows(
        obs_str, measurement_bases, measurement_outcomes
    )
    
    support = sum(1 for p in obs_str if p != 'I')
    var_bound = variance_bound(obs_str, NUM_SHADOWS)
    actual_var = result['variance']
    
    ci = compute_confidence_interval(
        result['expectation'], actual_var, NUM_SHADOWS
    )
    
    contains_truth = ci[0] <= expected <= ci[1]
    
    ci_str = f"[{ci[0]:>+6.2f}, {ci[1]:>+6.2f}]"
    
    print(f"{obs_str:<12} {support:<8} {var_bound:<12.2f} {actual_var:<12.2f} {ci_str:<25} {'Yes' if contains_truth else 'No'}")

print("\n" + "="*90)
print("Key Insight: Variance scales as 4^k, so observables with larger support are harder to estimate.")
print("More shadows are needed for high-weight observables.")

---

## Part 7: Scaling Up - More Shadows for Better Precision

Let's see how estimation improves with more shadows.

In [ ]:
# Run with more shadows using the QuartumSE implementation
SHADOW_SIZES = [10, 50, 100, 500, 1000]

results_by_size = {}

for shadow_size in SHADOW_SIZES:
    # Configure shadows
    config = ShadowConfig(
        version=ShadowVersion.V0_BASELINE,
        shadow_size=shadow_size,
        random_seed=RANDOM_SEED,
        confidence_level=0.95,
    )
    
    # Create shadow implementation
    shadow_impl = RandomLocalCliffordShadows(config)
    
    # Generate circuits
    circuits = shadow_impl.generate_measurement_circuits(bell_circuit, shadow_size)
    
    # Execute
    transpiled = transpile(circuits, backend=backend)
    job = backend.run(transpiled, shots=1)
    result = job.result()
    
    # Extract outcomes
    outcomes_list = []
    for i in range(shadow_size):
        counts = result.get_counts(i)
        bitstring = list(counts.keys())[0].replace(" ", "")
        outcomes = np.array([int(b) for b in bitstring[::-1]], dtype=int)
        outcomes_list.append(outcomes)
    
    measurement_outcomes_scaled = np.array(outcomes_list)
    
    # Reconstruct
    shadow_impl.reconstruct_classical_shadow(
        measurement_outcomes_scaled, shadow_impl.measurement_bases
    )
    
    # Estimate observables
    estimates = {}
    for obs_str in GROUND_TRUTH.keys():
        obs = Observable(obs_str, coefficient=1.0)
        est = shadow_impl.estimate_observable(obs)
        estimates[obs_str] = {
            'value': est.expectation_value,
            'ci': est.confidence_interval,
            'ci_width': est.ci_width,
        }
    
    results_by_size[shadow_size] = estimates

print("Estimation Precision vs Shadow Size:")
print("=" * 80)

In [ ]:
# Visualize scaling
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Error vs shadow size
for obs_str, expected in GROUND_TRUTH.items():
    errors = [abs(results_by_size[s][obs_str]['value'] - expected) for s in SHADOW_SIZES]
    axes[0].plot(SHADOW_SIZES, errors, 'o-', label=obs_str, markersize=8)

axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlabel('Shadow Size (M)', fontsize=12)
axes[0].set_ylabel('Absolute Error', fontsize=12)
axes[0].set_title('Estimation Error vs Shadow Size', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Add theoretical 1/sqrt(M) line
theoretical_scaling = [1/np.sqrt(s) for s in SHADOW_SIZES]
axes[0].plot(SHADOW_SIZES, theoretical_scaling, 'k--', alpha=0.5, label='1/sqrt(M)')

# Right: CI width vs shadow size
for obs_str in GROUND_TRUTH.keys():
    ci_widths = [results_by_size[s][obs_str]['ci_width'] for s in SHADOW_SIZES]
    axes[1].plot(SHADOW_SIZES, ci_widths, 'o-', label=obs_str, markersize=8)

axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel('Shadow Size (M)', fontsize=12)
axes[1].set_ylabel('95% CI Width', fontsize=12)
axes[1].set_title('Confidence Interval Width vs Shadow Size', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observations:")
print("- Error decreases roughly as 1/sqrt(M) (standard deviation scaling)")
print("- CI width shrinks with more shadows")
print("- Higher-weight observables (like YY) may have more variance")

---

## Bonus: Noise-Aware Shadows (v1) with MEM

On real hardware, measurement errors corrupt our outcomes. The v1 implementation adds **Measurement Error Mitigation (MEM)**.

### How MEM Works

1. **Calibration**: Prepare all $2^n$ computational basis states, measure each many times
2. **Build Confusion Matrix**: $A_{ij}$ = probability of measuring state $i$ when state $j$ was prepared
3. **Correction**: For each shadow, apply $A^{-1}$ to correct the measurement distribution

### In QuartumSE

The `NoiseAwareRandomLocalCliffordShadows` class:
1. Inherits from `RandomLocalCliffordShadows`
2. Takes a `MeasurementErrorMitigation` instance
3. Overrides `reconstruct_classical_shadow` to apply MEM correction
4. Overrides `_pauli_expectation_single_shadow` to use corrected distributions

In [ ]:
# Demonstrate MEM calibration (on ideal simulator, matrix should be ~identity)
from quartumse.mitigation import MeasurementErrorMitigation

# Create MEM instance
mem = MeasurementErrorMitigation(backend)

# Calibrate for 2 qubits
mem.calibrate(qubits=[0, 1], shots=1024)

print("Measurement Error Mitigation Calibration:")
print("=" * 60)
print("\nConfusion Matrix (rows=measured, cols=prepared):")
print("States: |00>, |01>, |10>, |11>")
print()
print(np.round(mem.confusion_matrix, 4))

print("\nInterpretation:")
print("- Diagonal elements = probability of correct measurement")
print("- Off-diagonal = probability of measurement errors")
print("- On ideal simulator, this should be ~identity matrix")

# Check if close to identity
is_ideal = np.allclose(mem.confusion_matrix, np.eye(4), atol=0.05)
print(f"\nClose to identity: {is_ideal} (expected on noiseless simulator)")

In [ ]:
# Show the v1 workflow
from quartumse.shadows.v1_noise_aware import NoiseAwareRandomLocalCliffordShadows

# Configure v1 shadows
config_v1 = ShadowConfig(
    version=ShadowVersion.V1_NOISE_AWARE,
    shadow_size=100,
    random_seed=RANDOM_SEED,
    apply_inverse_channel=True,
)

# Create v1 implementation with MEM
shadow_v1 = NoiseAwareRandomLocalCliffordShadows(config_v1, mem)

# Generate circuits
circuits_v1 = shadow_v1.generate_measurement_circuits(bell_circuit, 100)

# Execute
transpiled_v1 = transpile(circuits_v1, backend=backend)
job_v1 = backend.run(transpiled_v1, shots=1)
result_v1 = job_v1.result()

# Extract outcomes
outcomes_v1 = []
for i in range(100):
    counts = result_v1.get_counts(i)
    bitstring = list(counts.keys())[0].replace(" ", "")
    outcomes = np.array([int(b) for b in bitstring[::-1]], dtype=int)
    outcomes_v1.append(outcomes)

outcomes_v1 = np.array(outcomes_v1)

# Reconstruct (this applies MEM correction internally)
shadow_v1.reconstruct_classical_shadow(outcomes_v1, shadow_v1.measurement_bases)

print("Noise-Aware Shadows (v1) Results:")
print("=" * 60)
print(f"{'Observable':<12} {'v1 Estimate':<15} {'Expected':<10} {'Error'}")
print("-" * 50)

for obs_str, expected in GROUND_TRUTH.items():
    obs = Observable(obs_str, coefficient=1.0)
    est = shadow_v1.estimate_observable(obs)
    err = abs(est.expectation_value - expected)
    print(f"{obs_str:<12} {est.expectation_value:>+12.4f}   {expected:>+8.1f}   {err:.4f}")

print("\nNote: On ideal simulator, v0 and v1 should give similar results.")
print("On real hardware with noise, v1 should show improved accuracy.")

---

## QuartumSE Architecture Summary

### Class Hierarchy

```
ClassicalShadows (ABC)              # Abstract base class
├── generate_measurement_circuits() # Step 1-2: Basis selection + circuit generation
├── reconstruct_classical_shadow()  # Step 4: Inverse channel
├── estimate_observable()           # Step 5: Expectation estimation
└── compute_variance_bound()        # Step 6: Theoretical bounds

RandomLocalCliffordShadows          # v0 baseline implementation
├── basis_gates: {0: Z, 1: X, 2: Y} # Gate mapping
├── inverse_channel: 3|b><b| - I    # Reconstruction formula
└── _pauli_expectation_single_shadow() # Core estimator

NoiseAwareRandomLocalCliffordShadows # v1 with MEM
├── mem: MeasurementErrorMitigation # Confusion matrix correction
├── noise_corrected_distributions   # Corrected probability distributions
└── _pauli_expectation_single_shadow() # Uses corrected distributions
```

### Data Flow

```
State Preparation Circuit
         ↓
generate_measurement_circuits(circuit, num_shadows)
         ↓
[Circuit with random rotations + measurements] × num_shadows
         ↓
Backend Execution (shots=1 per circuit)
         ↓
measurement_outcomes: (num_shadows, num_qubits)
measurement_bases: (num_shadows, num_qubits)
         ↓
reconstruct_classical_shadow(outcomes, bases)
         ↓
[Shadow snapshots stored internally]
         ↓
estimate_observable(observable) × num_observables
         ↓
ShadowEstimate(expectation, variance, CI, ...)
```

---

## Summary: The 6 Steps of Classical Shadows

| Step | What Happens | Key Formula |
|------|--------------|-------------|
| **1. Basis Selection** | Sample random basis per qubit | $b_q \sim \text{Uniform}(X, Y, Z)$ |
| **2. Circuit Generation** | Add rotation gates + measurements | $H$ for X, $S^\dagger H$ for Y |
| **3. Execution** | Run circuits, collect outcomes | 1 shot per shadow |
| **4. Inverse Channel** | Apply correction factor | $\hat{\rho}_q = 3\|b_q\rangle\langle b_q\| - I$ |
| **5. Estimation** | Average compatible shadows | $\hat{o} = \frac{1}{M}\sum_i 3^k \cdot \text{sign}$ |
| **6. Confidence** | Compute variance and CI | $\text{Var} \leq 4^k / M$ |

### Key Takeaways

1. **Random Bases**: The randomization is crucial - it distributes information across all observables
2. **Inverse Channel**: The $3^k$ factor corrects for the "dilution" from random measurement
3. **Compatibility**: Only shadows measured in the right basis contribute (others give 0)
4. **Scaling**: Variance scales as $4^k/M$, so more shadows needed for high-weight observables
5. **Noise Mitigation**: v1 adds MEM to correct measurement errors on real hardware

---

*QuartumSE - Shot-efficient quantum measurement optimization*